In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.document_loaders import PyPDFLoader
import pdfplumber
import pandas as pd

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("ai_industry.pdf")

# 페이지 별 문서 로드
docs = loader.load()

# PDF 내부의 표 로드
with pdfplumber.open("ai_industry.pdf") as pdf:
    all_tables = []  
    
    for page_num, page in enumerate(pdf.pages):
        tables = page.extract_tables()  
        
        for table in tables:
            df = pd.DataFrame(table[1:], columns=table[0])
            all_tables.append((page_num, df)) 

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

In [11]:
while True: 
    print("========================")
    query = input("질문을 입력하세요: ")
    if query == "exit":
        break
    response = rag_chain_debug.invoke(query)

    print("Final Response:")
    print(response.content)

Debug Output: neurlPS의 개최장소는?
Debug Output: {'context': [Document(metadata={'source': 'ai_industry.pdf', 'page': 23}, page_content='21\n행사명행사 주요 개요\nNeurIPS 2024 \n-신경정보처리시스템재단은 인공지능과 머신러닝 분야의 연구 성과 교환을 촉진하는 것을 목적으로 하는 비영리 법인으로 매년 학제간 학술대회(NeurIPS)를 주최-이번 제38회 연례학술대회는 AI 연구자를 위한 실험 설계,  LLM을 위한 메타 생성 알고리즘, 정렬에 대한 학제 간 통찰력 등을 다룰 예정기간장소 홈페이지2024.12.10~15캐나다 밴쿠버 https://neurips.cc/ GenAI Summit Maroc 2024 \n-GenAI Summit Maroc 2024는 인공지능과 데이터 분석에 초점을 맞춘 최고의 이벤트로, 250명 이상의 업계 리더, 정책 입안자, 전문가가 모여 AI 발전을 탐구-이번 행사에는 오픈소스 AI, AI 주도 사이버 보안, 우수한 의사결정을 위한 생성AI와 예측 AI 결합 등을 다룰 예정기간장소 홈페이지2024.12.10~11모로코 https://genaimaroc.com/ \nAI Summit Seoul 2024\n-AI Summit Seoul 행사는 2018년 개최를 시작으로 금년도는 7회 행사로 개최-이번 행사는 AI와 산업의 융합에 초점을 두고 다양한 글로벌 기업과 기관, 학계 전문가 등 전문가들이 한자리에 모여 AI 및 산업 트렌드 등에 대한 주제 발표 및 워크샵 진행 기간장소 홈페이지2024.12.10~11서울(코엑스 그랜드볼룸) https://aisummit.co.kr/\nⅡ. 주요 행사 일정'), Document(metadata={'source': 'ai_industry.pdf', 'page': 1}, page_content='Ⅱ. 주요 행사  ▹NeurIPS 2024 ·····························

# 문제점
1. cb인사이츠는 스타트업이 실패할 것으로 예상함 ->챗봇대답: 긍정적으로 평가하고 있다
2. 카나나는 나다운 ai 또는 카(카오의 사명)+na(tive)+na(tural) ->챗봇대답: natural
3. 노벨상 수상자: 물리학상 수상자와 화학상 수상자가 있음 ->챗봇대답: 물리학상 수상자만 언급
4. gaira는 미국무부가 2024년 9월 23일 발표 ->챗봇대답: 미국무부가 23년 10월 30일 발표
5. Ori Goshen은 기업이 아니라 이스라엘 스타트업 AI12의 ceo ->챗봇대답: 오리고센은 미국기업입니다.
6. 가트너는 시장조사기관 ->챗봇대답: 글로벌 리서치 회사입니다 ?? hallucination??

# 개선방안
* 청크사이즈 늘리기
* 프롬프트엔지니어링


**pdf plumber 적용 및 청크사이즈 증가로 많은 부분 해결됨**